# Bostrom Revenue and Price Model

In [1]:
from src.data_extractors import get_pools, get_prices


BASE_INVESTMINTING_AMOUNT_AMPERE = 100_000_000
BASE_INVESTMINTING_AMOUNT_VOLT = 1_000_000_000
BASE_INVESTMINTING_PERIOD_AMPERE = 30
BASE_INVESTMINTING_PERIOD_VOLT = 30

NUMBER_DEGEN_ITERATIONS = 3
BOOT_REVENUE = 0.20  # stake + restake

## Optimal State
PRICE_HYDROGEN_OPTIMAL = 0.2
HYDROGEN_REVENUE_OPTIMAL = 0.4

## Actual State

In [2]:
price_df = get_prices(get_pools())

price_ampere_actual = price_df.loc['hydrogen', 'milliampere'] * 1000
price_volt_actual = price_df.loc['hydrogen', 'millivolt'] * 1000
price_hydrogen_actual = price_df.loc['boot', 'hydrogen']

print(f'Price Ampere in Hydrogen actual: {price_ampere_actual:>,.1f}\n'
      f'Price Volt in H actual:   {price_volt_actual:>,.1f}\n'
      f'Price H in BOOT actual:   {price_hydrogen_actual:>,.3f}')

Price Ampere in Hydrogen actual: 3,930,918.6
Price Volt in H actual:   36,280,521.2
Price H in BOOT actual:   0.374


## Degen
Stake BOOT -> Buy BOOT for HYDROGEN -> Stake BOOT - 3 times

In [3]:
degen_revenue_actual = degen_revenue_optimal = 0
staked_boot_actual = staked_boot_optimal = 1

for _ in range(NUMBER_DEGEN_ITERATIONS):
    degen_revenue_actual +=  staked_boot_actual * BOOT_REVENUE
    staked_boot_actual *= price_hydrogen_actual

    degen_revenue_optimal +=  staked_boot_optimal * BOOT_REVENUE
    staked_boot_optimal *= PRICE_HYDROGEN_OPTIMAL

print(f'Degen revenue actual {degen_revenue_actual:.1%}\n'
      f'Degen revenue optimal {degen_revenue_optimal:.1%}')

Degen revenue actual 30.3%
Degen revenue optimal 24.8%


## Staking + Investminting

Stake BOOT -> Investmint HYDROGEN

In [4]:
hydrogen_revenue_actual = \
    (1 * 365 / (BASE_INVESTMINTING_AMOUNT_VOLT * BASE_INVESTMINTING_PERIOD_VOLT) * price_volt_actual +
     1 * 365 / (BASE_INVESTMINTING_AMOUNT_AMPERE * BASE_INVESTMINTING_PERIOD_AMPERE) * price_ampere_actual) / 2
staking_investminting_revenue_actual = BOOT_REVENUE + price_hydrogen_actual * hydrogen_revenue_actual
staking_investminting_revenue_optimal = BOOT_REVENUE + PRICE_HYDROGEN_OPTIMAL * HYDROGEN_REVENUE_OPTIMAL

print(f'Staking + Investminting revenue actual {staking_investminting_revenue_actual:.1%}\n'
      f'Staking + Investminting revenue optimal {staking_investminting_revenue_optimal:.1%}')

Staking + Investminting revenue actual 37.2%
Staking + Investminting revenue optimal 28.0%


## Investminting
Buy HYDROGEN for BOOT -> Investmint HYDROGEN

In [5]:
print(f'Investminting revenue actual {hydrogen_revenue_actual:.1%}\n'
      f'Investminting revenue optimal {HYDROGEN_REVENUE_OPTIMAL:.1%}')

Investminting revenue actual 46.0%
Investminting revenue optimal 40.0%


## Prices

In [6]:
price_ampere_optimal = BASE_INVESTMINTING_AMOUNT_AMPERE * BASE_INVESTMINTING_PERIOD_AMPERE / 365 * HYDROGEN_REVENUE_OPTIMAL
price_volt_optimal = BASE_INVESTMINTING_AMOUNT_VOLT * BASE_INVESTMINTING_PERIOD_VOLT / 365 * HYDROGEN_REVENUE_OPTIMAL

print(f'AMPERE price actual {price_ampere_actual:>,.1f} optimal {price_ampere_optimal:>,.1f} can decrease by {price_ampere_actual/price_ampere_optimal:.2f} times')
print(f'VOLT price actual {price_volt_actual:>,.1f} optimal {price_volt_optimal:>,.1f} can decrease by {price_volt_actual/price_volt_optimal:.2f} times')
print(f'H price actual {price_hydrogen_actual:.3f} optimal {PRICE_HYDROGEN_OPTIMAL} can decrease by {price_hydrogen_actual/PRICE_HYDROGEN_OPTIMAL:.2f} times')

AMPERE price actual 3,930,918.6 optimal 3,287,671.2 can decrease by 1.20 times
VOLT price actual 36,280,521.2 optimal 32,876,712.3 can decrease by 1.10 times
H price actual 0.374 optimal 0.2 can decrease by 1.87 times


In [7]:
print(f"OSMO price"
      f"\n\t- in bostrom {price_df.loc['hydrogen', 'uosmo in bostrom'] * price_df.loc['boot', 'hydrogen'] * 1_000_000:>,.0f} BOOT"
      f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uosmo'] * 1_000_000:>,.0f} BOOT")
print(f"ATOM price"
      f"\n\t- in bostrom {price_df.loc['hydrogen', 'uatom in bostrom'] * price_df.loc['boot', 'hydrogen'] * 1_000_000:>,.0f} BOOT"
      f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uatom in osmosis'] * 1_000_000:>,.0f} BOOT")

OSMO price
	- in bostrom 89,813,118 BOOT
	- in osmosis 86,051,870 BOOT
ATOM price
	- in bostrom 1,180,074,028 BOOT
	- in osmosis 1,178,579,378 BOOT
